# Descripción del proyecto  

Rusty Bargain es un servicio de venta de coches de segunda mano que está desarrollando una app para atraer a nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial, especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.  
  
## A Rusty Bargain le interesa:
  
* la calidad de la predicción  
* la velocidad de la predicción  
* el tiempo requerido para el entrenamiento  
  
### Instrucciones del proyecto  

1. Descarga y examina los datos.  
2. Entrena diferentes modelos con varios hiperparámetros (debes hacer al menos dos modelos diferentes, pero más es mejor. Recuerda, varias implementaciones de potenciación del gradiente no cuentan como modelos diferentes). El punto principal de este paso es comparar métodos de potenciación del gradiente con bosque aleatorio, árbol de decisión y regresión lineal.  
3. Analiza la velocidad y la calidad de los modelos.  
  
  
Observaciones:  
  
* Utiliza la métrica RECM para evaluar los modelos.  
* La regresión lineal no es muy buena para el ajuste de hiperparámetros, pero es perfecta para hacer una prueba de cordura de otros métodos. Si la potenciación del gradiente funciona peor que la regresión lineal, definitivamente algo salió mal.  
* Aprende por tu propia cuenta sobre la librería LightGBM y sus herramientas para crear modelos de potenciación del gradiente (gradient boosting).  
* Idealmente, tu proyecto debe tener regresión lineal para una prueba de cordura, un algoritmo basado en árbol con ajuste de hiperparámetros (preferiblemente, bosque aleatorio), LightGBM con ajuste de hiperparámetros (prueba un par de conjuntos), y CatBoost y XGBoost con ajuste de hiperparámetros (opcional).  
* Toma nota de la codificación de características categóricas para algoritmos simples. LightGBM y CatBoost tienen su implementación, pero XGBoost requiere OHE.  
* Puedes usar un comando especial para encontrar el tiempo de ejecución del código de celda en Jupyter Notebook. Encuentra ese comando.  
* Dado que el entrenamiento de un modelo de potenciación del gradiente puede llevar mucho tiempo, cambia solo algunos parámetros del modelo.
Si Jupyter Notebook deja de funcionar, elimina las variables excesivas por medio del operador del:  
  
  del features_train  
   
### Descripción de los datos  
  
El dataset está almacenado en el archivo /datasets/car_data.csv. descargar dataset.  
  
### Características  
  
* DateCrawled — fecha en la que se descargó el perfil de la base de datos  
* VehicleType — tipo de carrocería del vehículo  
* RegistrationYear — año de matriculación del vehículo  
* Gearbox — tipo de caja de cambios  
* Power — potencia (CV)  
* Model — modelo del vehículo  
* Mileage — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)  
* RegistrationMonth — mes de matriculación del vehículo  
* FuelType — tipo de combustible  
* Brand — marca del vehículo  
* NotRepaired — vehículo con o sin reparación  
* DateCreated — fecha de creación del perfil  
* NumberOfPictures — número de fotos del vehículo  
* PostalCode — código postal del propietario del perfil (usuario)  
* LastSeen — fecha de la última vez que el usuario estuvo activo  
  
### Objetivo  

Price — precio (en euros)  
  
# Evaluación del proyecto  
  
Hemos definido los criterios de evaluación para el proyecto. Léelos con atención antes de pasar al ejercicio.  
  
Esto es en lo que se fijarán los revisores al examinar tu proyecto:  
  
* ¿Seguiste todos los pasos de las instrucciones?
* ¿Cómo preparaste los datos?
* ¿Qué modelos e hiperparámetros consideraste?
* ¿Conseguiste evitar la duplicación del código?
* ¿Cuáles son tus hallazgos?
* ¿Mantuviste la estructura del proyecto?
* ¿Mantuviste el código ordenado?

# Preparación de Datos

In [58]:
# Importar librerías
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb

from IPython.display import display
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from timeit import default_timer as timer

In [59]:
# Cargar los datos
df = pd.read_csv('datasets/car_data.csv')

In [60]:
# Exploración inicial
print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


Revisión y limpieza:  

Eliminar columnas que probablemente no serán útiles para el modelo, como DateCrawled, DateCreated, LastSeen, PostalCode, y NumberOfPictures.  

Manejar valores nulos en columnas relevantes como VehicleType, Gearbox, FuelType, y NotRepaired.  

Convertir valores categóricos a tipo category para optimizar la memoria.  

Analizar y tratar posibles errores en RegistrationYear y Power.  

In [61]:
# Eliminar columnas innecesarias
df.drop(['DateCrawled', 'DateCreated', 'LastSeen', 'PostalCode', 'NumberOfPictures'], axis=1, inplace=True)

In [62]:
# Rellenar valores faltantes en columnas categóricas con un valor específico o "Unknown"
df['VehicleType'].fillna('Unknown', inplace=True)
df['Gearbox'].fillna('Unknown', inplace=True)
df['FuelType'].fillna('Unknown', inplace=True)
df['NotRepaired'].fillna('Unknown', inplace=True)


In [63]:
# Tratar errores en 'RegistrationYear' y 'Power'
df = df[(df['RegistrationYear'] >= 1886) & (df['RegistrationYear'] <= 2024)]
df = df[df['Power'].between(1, 1000)]

In [64]:
# Convertir columnas categóricas
for col in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']:
    df[col] = df[col].astype('category')

In [65]:
# Confirmar limpieza
print(df.info())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 313805 entries, 1 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Price              313805 non-null  int64   
 1   VehicleType        313805 non-null  category
 2   RegistrationYear   313805 non-null  int64   
 3   Gearbox            313805 non-null  category
 4   Power              313805 non-null  int64   
 5   Model              300445 non-null  category
 6   Mileage            313805 non-null  int64   
 7   RegistrationMonth  313805 non-null  int64   
 8   FuelType           313805 non-null  category
 9   Brand              313805 non-null  category
 10  NotRepaired        313805 non-null  category
dtypes: category(6), int64(5)
memory usage: 16.5 MB
None


,Price,RegistrationYear,Power,Mileage,RegistrationMonth
count,313805.000000,313805.000000,313805.000000,313805.000000,313805.000000
mean,4705.696209,2003.247240,120.289215,128505.520945,5.934462
std,4590.966346,7.007376,54.914872,36805.430101,3.615522
min,0.000000,1910.000000,1.000000,5000.000000,0.000000
25%,1250.000000,1999.000000,75.000000,125000.000000,3.000000
50%,2999.000000,2003.000000,110.000000,150000.000000,6.000000
75%,6890.000000,2008.000000,150.000000,150000.000000,9.000000
max,20000.000000,2019.000000,1000.000000,150000.000000,12.000000


# Entrenamiento del Modelo  

In [66]:
#  Dividir los datos en conjuntos de entrenamiento y prueba
# Definir características y objetivo
X = df.drop('Price', axis=1)
y = df['Price']

In [67]:
# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
# Preprocesamiento y codificación para modelos
categorical_features = X.select_dtypes(include=['category']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', min_frequency=0.05), categorical_features)], remainder='passthrough')

### Definir y entrenar modelos

Modelos a considerar:  
Regresión lineal (prueba de cordura)  
Árbol de decisión  
Bosque aleatorio  
LightGBM (recomendado para datos de alta dimensionalidad y muchas categorías)  

In [69]:
# Configuración de modelos y ajustes de hiperparámetros básicos
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1),
    'LightGBM': lgb.LGBMRegressor(n_estimators=50, max_depth=10, learning_rate=0.1, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=50, max_depth=10, learning_rate=0.1, random_state=42)
}

In [70]:
# Entrenar y evaluar cada modelo
results = {}
for model_name, model in models.items():
    print(f"\nEntrenando modelo: {model_name}")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])
    
    start_time = timer()  # Tiempo de inicio
    pipeline.fit(X_train, y_train)
    end_time = timer()  # Tiempo final

    # Predicciones y evaluación
    y_pred = pipeline.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    results[model_name] = {'RMSE': rmse, 'R2': r2, 'Train Time (s)': end_time - start_time}

    print(f"{model_name} - RMSE: {rmse:.4f}, R2: {r2:.4f}, Train Time: {end_time - start_time:.2f} segundos")




Entrenando modelo: Linear Regression
Linear Regression - RMSE: 2890.0177, R2: 0.6054, Train Time: 2.50 segundos

Entrenando modelo: Decision Tree
Decision Tree - RMSE: 2095.2619, R2: 0.7926, Train Time: 2.21 segundos

Entrenando modelo: Random Forest
Random Forest - RMSE: 1994.3981, R2: 0.8121, Train Time: 23.14 segundos

Entrenando modelo: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008199 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 408
[LightGBM] [Info] Number of data points in the train set: 251044, number of used features: 32
[LightGBM] [Info] Start training from score 4702.491643
LightGBM - RMSE: 1925.8205, R2: 0.8248, Train Time: 1.41 segundos

Entrenando modelo: XGBoost
XGBoost - RMSE: 1772.3397, R2: 0.8516, Train Time: 2.63 segundos


# Análisis del Modelo  

Objetivo del Análisis:  

El análisis del modelo busca evaluar y comparar la precisión y el rendimiento de los modelos de predicción de precio de vehículos de segunda mano, con énfasis en la métrica RMSE (raíz cuadrada del error cuadrático medio) y el coeficiente de determinación 𝑅<sup>2</sup>. Estos resultados ayudarán a determinar el modelo que mejor equilibra precisión y eficiencia para ser implementado en la app de Rusty Bargain.

In [71]:
# Resumen de resultados
results_df = pd.DataFrame(results).T
display(results_df.sort_values(by='RMSE'))

,RMSE,R2,Train Time (s)
XGBoost,1772.339650,0.851592,2.629884
LightGBM,1925.820513,0.824775,1.407804
Random Forest,1994.398127,0.812074,23.136102
Decision Tree,2095.261909,0.792585,2.213215
Linear Regression,2890.017744,0.605393,2.495453


In [72]:
# Medición de tiempo de predicción para el mejor modelo
best_model_name = results_df['RMSE'].idxmin()
best_model = models[best_model_name]

### Conclusiones del Análisis de Modelos y Recomendaciones  
  
Linear Regression:

RMSE: 2890.02  
𝑅<sup>2</sup>: 0.6054  
Train Time: 2.50 segundos  
Análisis: La regresión lineal tiene el peor rendimiento en términos de error (RMSE) y coeficiente de determinación (𝑅<sup>2</sup>), lo cual es esperado, ya que este modelo es lineal y no captura bien las relaciones complejas. Sin embargo, es una buena base para comparar otros modelos.  
  
Decision Tree:  

RMSE: 2095.26  
𝑅<sup>2</sup>: 0.7926  
Train Time: 2.21 segundos  
Análisis: El árbol de decisión mejora considerablemente el rendimiento con respecto a la regresión lineal, reduciendo el RMSE y aumentando 𝑅<sup>2</sup>, lo cual indica un mejor ajuste. Aunque es un modelo rápido y simple, su rendimiento es inferior al de los otros modelos más avanzados.  

Random Forest:  

RMSE: 1994.40  
𝑅<sup>2</sup>: 0.8121  
Train Time: 23.14 segundos  
Análisis: El bosque aleatorio muestra una mejora significativa en comparación con el árbol de decisión, y alcanza un buen equilibrio entre RMSE y 𝑅<sup>2</sup>. Sin embargo, el tiempo de entrenamiento es considerablemente mayor debido al ensamblado de múltiples árboles.  

LightGBM:  

RMSE: 1925.82  
𝑅<sup>2</sup>: 0.8248  
Train Time: 1.41 segundos  
Análisis: LightGBM tiene un rendimiento excelente, logrando un RMSE bajo y un 𝑅<sup>2</sup> más alto en un tiempo de entrenamiento muy corto. Este modelo es eficiente y adecuado para grandes conjuntos de datos, mostrando una combinación de precisión y velocidad.

XGBoost:  

RMSE: 1772.34  
𝑅<sup>2</sup>: 0.8516  
Train Time: 2.63 segundos  
Análisis: XGBoost es el mejor en términos de precisión, obteniendo el RMSE más bajo y el 𝑅<sup>2</sup> más alto entre todos los modelos. Aunque el tiempo de entrenamiento es mayor que LightGBM, el incremento en precisión lo convierte en la mejor elección para este conjunto de datos.


Modelo Seleccionado   
  
Conclusión General:  

Mejor Modelo: XGBoost, ya que obtiene el mejor RMSE y R<sup>2</sup>, lo que indica una precisión superior en las predicciones.  

Modelos Alternativos: LightGBM es una excelente alternativa si el tiempo de entrenamiento es un factor importante, ya que ofrece una precisión cercana a XGBoost en menos tiempo.  

Recomendación: Considerar el uso de XGBoost para este caso, ya que su mayor precisión beneficia la predicción del valor de los autos usados, aunque LightGBM podría usarse si se requiere un modelo con un balance óptimo entre precisión y velocidad.